<a href="https://colab.research.google.com/github/caresmo5/3072game/blob/main/prophet_todos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from datetime import datetime, timedelta, time

In [77]:
pais = input('¿Qué país? (AT / NL / IT) ')

# Festivos 2022, 2023 y 2024 según ChatGPT

festivos_AT = ['2022-01-01', '2022-01-06', '2022-04-18', '2022-05-01', '2022-05-26', '2022-06-06', '2022-06-16', '2022-10-26', '2022-11-01', '2022-12-08', '2022-12-25', '2022-12-26',
              '2023-01-01', '2023-01-06', '2023-04-10', '2023-05-01', '2023-05-18', '2023-06-05', '2023-06-15', '2023-10-26', '2023-11-01', '2023-12-08', '2023-12-25', '2023-12-26',
              '2024-01-01', '2024-01-06', '2024-04-01', '2024-05-01', '2024-05-09', '2024-05-20', '2024-05-30', '2024-10-26', '2024-11-01', '2024-12-08', '2024-12-25', '2024-12-26']

festivos_NL = ['2022-04-15', '2022-04-17', '2022-04-27', '2022-05-05', '2022-05-26', '2022-06-05', '2022-12-25', '2022-12-26',
                            '2023-04-07', '2023-04-09', '2023-04-27', '2023-05-05', '2023-05-18', '2023-05-28', '2023-12-25', '2023-12-26',
                            '2024-03-29', '2024-03-31', '2024-04-27', '2024-05-05', '2024-05-09', '2024-05-19', '2024-12-25', '2024-12-26']

festivos_IT = ['2022-01-01', '2022-01-06', '2022-04-15', '2022-04-17', '2022-04-25', '2022-05-01', '2022-06-02', '2022-08-15', '2022-11-01', '2022-12-08', '2022-12-25', '2022-12-26',
              '2023-01-01', '2023-01-06', '2023-04-15', '2023-04-17', '2023-04-25', '2023-05-01', '2023-06-02', '2023-08-15', '2023-11-01', '2023-12-08', '2023-12-25', '2023-12-26',
              '2024-01-01', '2024-01-06', '2024-04-06', '2024-04-08', '2024-04-25', '2024-05-01', '2024-06-02', '2024-08-15', '2024-11-01', '2024-12-08', '2024-12-25', '2024-12-26']

if pais == 'AT':
  festivos = pd.DataFrame({

      'holiday': 'festivos',
      'ds': pd.to_datetime(festivos_AT),
      'lower_window': 0,
      'upper_window': 1,
  })
elif pais == 'NL':
  festivos = pd.DataFrame({
      'holiday': 'festivos',
      'ds': pd.to_datetime(festivos_NL),
      'lower_window': 0,
      'upper_window': 1,
  })
elif pais == 'IT':
  festivos = pd.DataFrame({
      'holiday': 'festivos',
      'ds': pd.to_datetime(festivos_IT),
      'lower_window': 0,
      'upper_window': 1,
  })
else:
  print("No entiendo.")

df = pd.read_excel('/content/Histórico.xlsx', sheet_name='Train')
df = df.loc[:,['Fecha',pais]]
df.columns = ['ds','y']
df['ds'] = pd.to_datetime(df.ds)

df['festivos'] = df['ds'].isin(festivos['ds']).astype(int)

if pais == "IT" or pais == "NL":
  df = df[df['ds'].dt.dayofweek != 6]

df = df.dropna(subset='y')

df

¿Qué país? (AT / NL / IT) IT


,ds,y,festivos
1,2022-03-14,601.0,0
2,2022-03-15,390.0,0
3,2022-03-16,400.0,0
4,2022-03-17,358.0,0
5,2022-03-18,308.0,0
...,...,...,...
744,2024-03-26,433.0,0
745,2024-03-27,436.0,0
746,2024-03-28,387.0,0
747,2024-03-29,363.0,0


In [81]:
inicio = df.iloc[-1,0]
fin = input('¿Hasta qué día? (aaaa-mm-dd) ')
fin = datetime.strptime(fin,'%Y-%m-%d')
diferencia = fin - inicio
diferencia = diferencia.days

model = Prophet()
model = Prophet(changepoint_prior_scale=0.5)
model = Prophet(seasonality_prior_scale=10)
model.add_regressor('festivos')
model.fit(df)
future = model.make_future_dataframe(periods=diferencia)
future['festivos'] = future['ds'].isin(festivos['ds']).astype(int)
if pais == "IT" or pais == "NL":
  future = future[future['ds'].dt.dayofweek != 6]
pred = model.predict(future)
pred = pred.loc[:,['ds','yhat']]
pred['yhat'] = abs(pred['yhat'])
pred

¿Hasta qué día? (aaaa-mm-dd) 2024-06-10


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjb8a0_wi/8cfcmm76.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjb8a0_wi/0sk0rfpv.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=10765', 'data', 'file=/tmp/tmpjb8a0_wi/8cfcmm76.json', 'init=/tmp/tmpjb8a0_wi/0sk0rfpv.json', 'output', 'file=/tmp/tmpjb8a0_wi/prophet_model8w_hlnb3/prophet_model-20240611142544.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:25:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:25:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds,yhat
0,2022-03-14,601.866038
1,2022-03-15,440.982827
2,2022-03-16,400.923753
3,2022-03-17,372.312599
4,2022-03-18,348.453980
...,...,...
698,2024-06-05,489.424810
699,2024-06-06,463.456350
700,2024-06-07,441.705331
701,2024-06-08,150.104590


In [84]:
peso = pd.read_excel('/content/Pesos.xlsx', sheet_name='AT')
fechas = pd.date_range(start=inicio, end=fin, freq='D')

if pais == 'AT':
  fechas_repetidas = fechas.repeat(26)
  horas = pd.date_range(start='07:00', end='19:30', freq='30min').time
elif pais == 'NL':
  fechas_repetidas = fechas.repeat(24)
  horas = pd.date_range(start='08:00', end='19:30', freq='30min').time
elif pais == 'IT':
  fechas_repetidas = fechas.repeat(24)
  horas = pd.date_range(start='08:00', end='19:30', freq='30min').time


# Repetir cada hora para cada fecha
horas_repetidas = list(horas) * len(fechas)

# Crear un dataframe con las fechas y horas repetidas
porhoras = pd.DataFrame({'ds': fechas_repetidas, 'Hora': horas_repetidas})

# Convertir la columna de fechas a tipo datetime
porhoras['ds'] = pd.to_datetime(porhoras['ds'])

# Añadir una columna con el día de la semana
porhoras['Dia'] = porhoras['ds'].dt.day_name()

porhoras = pd.merge(porhoras, pred, on='ds', how='left')

porhoras = pd.merge(porhoras, peso, on=['Dia','Hora'], how='left')

porhoras['fc'] = porhoras['yhat'] * porhoras['Peso']

porhoras_pivot = porhoras.pivot(index='Hora', columns='ds', values='fc')
porhoras_pivot

ds,2024-03-30,2024-03-31,2024-04-01,2024-04-02,2024-04-03,2024-04-04,2024-04-05,2024-04-06,2024-04-07,2024-04-08,...,2024-06-01,2024-06-02,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,2024-06-09,2024-06-10
Hora,,,,,,,,,,,,,,,,,,,,,
08:00:00,3.208854,NaN,30.928225,19.623957,17.778055,16.656435,14.932972,16.243804,NaN,9.100372,...,6.559678,NaN,34.798359,22.948863,21.045267,19.882277,17.933236,6.784727,NaN,34.777077
08:30:00,3.932976,NaN,32.629582,20.074047,19.845270,18.131830,16.772008,19.909441,NaN,9.600982,...,8.039960,NaN,36.712611,23.475213,23.492391,21.643412,20.141763,8.315794,NaN,36.690158
09:00:00,4.841678,NaN,36.579159,24.349909,24.227768,23.489844,21.774186,24.509457,NaN,10.763112,...,9.897568,NaN,41.156409,28.475539,28.680294,28.039109,26.148956,10.237133,NaN,41.131238
09:30:00,5.587097,NaN,38.523566,28.445735,26.832459,25.469979,24.937328,28.282907,NaN,11.335238,...,11.421387,NaN,43.344125,33.265324,31.763670,30.402737,29.947621,11.813231,NaN,43.317616
10:00:00,5.501906,NaN,40.164159,29.976044,26.997837,26.013546,25.121231,27.851655,NaN,11.817969,...,11.247236,NaN,45.190010,35.054915,31.959440,31.051575,30.168474,11.633106,NaN,45.162373
10:30:00,5.388319,NaN,38.158989,29.931035,26.419016,26.285329,23.981029,27.276654,NaN,11.227964,...,11.015035,NaN,42.933928,35.002280,31.274245,31.375995,28.799188,11.392938,NaN,42.907671
11:00:00,4.834578,NaN,35.424667,27.455536,23.938358,22.402710,23.429318,24.473519,NaN,10.423413,...,9.883055,NaN,39.857452,32.107354,28.337697,26.741431,28.136630,10.222123,NaN,39.833076
11:30:00,4.763586,NaN,34.695514,25.925227,24.517178,22.286232,22.914388,24.114143,NaN,10.208866,...,9.737929,NaN,39.037059,30.317764,29.022891,26.602395,27.518242,10.072018,NaN,39.013184
12:00:00,3.698701,NaN,27.950852,20.119056,19.307794,18.287134,17.617965,18.723500,NaN,8.224305,...,7.561045,NaN,31.448419,23.527848,22.856139,21.828794,21.157685,7.820449,NaN,31.429185


In [85]:
porhoras_pivot.to_excel('predAT.xlsx')
files.download('predAT.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>